Simple word2vec model with 200 features for each utterance, not considering other features (speaker etc.)

Doc2Vec model for all chats

In [188]:
import os
import json
import gensim

# path = r"C:\Users\user\git\NLPProj\Frames-dataset\chats"


def build_label_dict():
    d = {}
    label_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split("\t")
        d[k] = v.strip()
    return d

def gen_chat_model(label_dict):
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats = []
    avg_chat_len = 0
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            for turn in chat['turns']:
                sents.extend( turn['text'].split() )
                avg_chat_len += 1
        tagged_chat = gensim.models.doc2vec.TaggedDocument(sents, label_dict[filename])
        chats.append( tagged_chat )
    avg_chat_len /= len(chats)
    print("average number of sents in chat: " + str(avg_chat_len))
    return gensim.models.Doc2Vec(chats, size=100, window=8, min_count=1, workers=4)

# d = build_label_dict()
# chat_model = gen_chat_model(d)

Prepare input vectors from first parts of chats and labels from dictionary

In [190]:
def infer_vectors(chat_model, label_dict, num_of_sents_from_start):
    '''infer vectors for every sentence in the chats. 
    Will be used to load the training data and tags'''
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats_vecs = []
    labels = []
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            lim = min(num_of_sents_from_start, len(chat['turns']))
            for i in range( lim ):
                sents.extend( chat['turns'][i]['text'].split() )
#             for turn in chat['turns']:
#                 sents.extend( turn['text'].split() )
        sent_vec = chat_model.infer_vector( sents )
        chats_vecs.append( sent_vec )
        labels.append(label_dict[filename])
    return np.array(chats_vecs), np.array(labels)

Define the RNN - TBD

In [191]:
import numpy as np
from keras.layers import Input, Dense, merge
from keras.models import Model
import keras
import sys
# should change to relative import
sys.path.insert(0, '/home/amirgilad/Documents/git/NLPProj/kerasAttention')
import attention_dense
import attention_lstm

input_dims = x_train.shape
train_size = 1000

# Build doc2Vec model for the chats
d = build_label_dict()
chat_model = gen_chat_model(d)

# Load the data for the train and test
# The last param is the num. of rows we take from the begining of the chat
num_of_rows_from_start_of_chat = 5
X, Y = infer_vectors(chat_model, d, num_of_rows_from_start_of_chat)

# Divide into training and test data
x_train, y_train = X[:train_size], Y[:train_size]
x_test, y_test = X[train_size:], Y[train_size:]

# one attention layer
# inputs = Input(shape=(input_dims))#(input_dims,)
# attention_probs = Dense(input_dims[1], activation='softmax', name='attention_probs')(inputs)
# attention_mul = merge([inputs, attention_probs], output_shape=(1,), name='attention_mul', mode='mul')

# define the model
# model = Model(input=inputs, output=attention_mul)
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(x_train, y_train)

# Define the model
model = Sequential()
model.add(Dense(80, input_dim=100, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(40, activation='softmax', name='attention_probs'))
model.add(Dense(1, activation='sigmoid'))

# Compile and fit
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=150, batch_size=10)

# Test the model
scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

average number of sents in chat: 14.598977355734112
Epoch 1/150
1000/1000 [==============================] - 0s - loss: 0.6819 - acc: 0.6140     
Epoch 2/150
1000/1000 [==============================] - 0s - loss: 0.6639 - acc: 0.6140     
Epoch 3/150
1000/1000 [==============================] - 0s - loss: 0.6501 - acc: 0.6140     
Epoch 4/150
1000/1000 [==============================] - 0s - loss: 0.6320 - acc: 0.6650     
Epoch 5/150
1000/1000 [==============================] - 0s - loss: 0.6143 - acc: 0.7050     
Epoch 6/150
1000/1000 [==============================] - 0s - loss: 0.6002 - acc: 0.7160     
Epoch 7/150
1000/1000 [==============================] - 0s - loss: 0.5849 - acc: 0.7410     
Epoch 8/150
1000/1000 [==============================] - 0s - loss: 0.5727 - acc: 0.7490     
Epoch 9/150
1000/1000 [==============================] - 0s - loss: 0.5603 - acc: 0.7550     
Epoch 10/150
1000/1000 [==============================] - 0s - loss: 0.5465 - acc: 0.7700     
Epoch 1

1000/1000 [==============================] - 0s - loss: 0.1713 - acc: 0.9580     
Epoch 86/150
1000/1000 [==============================] - 0s - loss: 0.1711 - acc: 0.9580     
Epoch 87/150
1000/1000 [==============================] - 0s - loss: 0.1710 - acc: 0.9580     
Epoch 88/150
1000/1000 [==============================] - 0s - loss: 0.1708 - acc: 0.9580     
Epoch 89/150
1000/1000 [==============================] - 0s - loss: 0.1708 - acc: 0.9580     
Epoch 90/150
1000/1000 [==============================] - 0s - loss: 0.1706 - acc: 0.9580     
Epoch 91/150
1000/1000 [==============================] - 0s - loss: 0.1705 - acc: 0.9580     
Epoch 92/150
1000/1000 [==============================] - 0s - loss: 0.1704 - acc: 0.9580     
Epoch 93/150
1000/1000 [==============================] - 0s - loss: 0.1703 - acc: 0.9580     
Epoch 94/150
1000/1000 [==============================] - 0s - loss: 0.1671 - acc: 0.9590     
Epoch 95/150
1000/1000 [==============================] - 0s - 